In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

import pandas as pd
from data_utils import load_dataset

In [ ]:
df = load_dataset("../outputs/batch_run/faith_template5/execution/Mistral-7B-Instruct-v0.2_rv16-ld0.05-bs1-p0-nta1-e3-template_engpeft-t0.2-topp0.95_executed.parquet.gzip")
df.head()

In [ ]:
print(df.iloc[0].translated_prompt)

In [ ]:
print(df.iloc[0].linked_query)

In [ ]:
print(df.iloc[0].target_raw)


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

print(f"{tokenizer.unk_token=}")
print(f"{tokenizer.pad_token=}")
print(f"{tokenizer.unk_token_id=}")
print(f"{tokenizer.pad_token_id=}")

tokenizer.pad_token = tokenizer.unk_token

print(f"{tokenizer.unk_token=}")
print(f"{tokenizer.pad_token=}")
print(f"{tokenizer.unk_token_id=}")
print(f"{tokenizer.pad_token_id=}")

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

In [ ]:
df = load_dataset("../outputs/batch_run/democracy_template8/execution/Mistral-7B-Instruct-v0.2_rv32-ld0.05-bs1-p0-nta1-e3-template_engpeft-t0.2-topp0.95_executed.parquet.gzip")
df.head()

In [ ]:
handpicked_data = df.loc[(~df['execution'].str.startswith("exception:")) & (~df['execution'].isnull()) & (df['execution'].map(len) > 0)].iloc[[1, 2, 3, 206, 207]]

In [ ]:
preds = tokenizer.batch_encode_plus([f"`sparql\n{i}`" for i in handpicked_data['translated_prompt'].to_list()], add_special_tokens=False, padding='longest', return_tensors='np')['input_ids'].tolist()
labels = tokenizer.batch_encode_plus([f"`sparql\n{i}`" for i in handpicked_data['target_template'].to_list()], add_special_tokens=False, padding='longest', return_tensors='np')['input_ids'].tolist()

In [ ]:
import json
from pathlib import Path

data = {
    "preds": preds,
    "labels": labels,
}

path = Path("sft_peft_compute_metrics_execute_ok.json")
path.write_text(json.dumps(data))

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset


df = load_dataset("../outputs/batch_run/lezgo/execution/Mistral-7B-Instruct-v0.2_rv16-ld0-bs1-p0-nta0-e3-template_engpeft-t0.2-topp0.95_executed.parquet.gzip")
df.head()

In [ ]:
import json
from pathlib import Path

df_out = df.head()

path = Path("evaluation_test.json")
path.write_text(df_out.to_json())

In [ ]:
import pandas as pd

data_gold = json.loads(Path("../outputs/batch_run/lezgo/preprocessed_gold.json").read_text())
df_gold = pd.read_json(data_gold['df_gold_eval'])
df_gold.head()

In [ ]:
df_gold_out = df_gold.head()

path = Path("evaluation_test_gold.json")
path.write_text(df_gold_out.to_json())

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

import pandas as pd

In [ ]:
dataset = load_dataset("../datasets/final_queries_v1.7.json")
dataset.head()

In [ ]:
df_out = dataset[2549:2553]
df_out.head()

In [ ]:
df_out_v2 = df_out.reset_index(drop=True)
df_out_v2.head()

In [ ]:
path = Path("templatize_test.json")
path.write_text(df_out_v2.to_json())

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

In [ ]:
df = load_dataset("../outputs/dataset_pipeline/fq17_3/fq17_3-split_test.pkl")
df.head()

In [ ]:
entry = df.iloc[0]
for c in df.columns:
    print(c)
    print(entry[c])
    print()

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

In [ ]:
df = load_dataset("../outputs/dataset_pipeline/fq17_3/fq17_3-split_test.pkl")
df.head()

In [ ]:
print(df.iloc[2].target_template)

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from datasets import load_dataset
from sft_peft import format_prompt
import sft_peft
from prompts_template import BASE_MISTRAL_TEMPLATE, BASE_BASIC_INSTRUCTION
from libwikidatallm.TemplateLLMQuerySender import TemplateLLMQuerySender

In [2]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.unk_token
collator = DataCollatorForCompletionOnlyLM(response_template="[/INST]", tokenizer=tokenizer)
datasets = load_dataset("pandas", data_files=
                          {
                              "valid": "../outputs/dataset_pipeline/fq17_3/fq17_3-split_valid.pkl",
                              "train": "../outputs/dataset_pipeline/fq17_3/fq17_3-split_train.pkl",
                              "test": "../outputs/dataset_pipeline/fq17_3/fq17_3-split_test.pkl",
                          })

In [4]:
datasets

DatasetDict({
    valid: Dataset({
        features: ['basic_input', 'templated_input', 'target_raw', 'target_template', 'index'],
        num_rows: 129
    })
    train: Dataset({
        features: ['basic_input', 'templated_input', 'target_raw', 'target_template', 'index'],
        num_rows: 1923
    })
    test: Dataset({
        features: ['basic_input', 'templated_input', 'target_raw', 'target_template', 'index'],
        num_rows: 513
    })
})

In [5]:
# Taken from SFTTrainer(Trainer) _prepare_non_packed_dataloader method
def _prepare_non_packed_dataloader(
    tokenizer,
    dataset,
    dataset_text_field,
    max_seq_length,
    formatting_func=None,
    add_special_tokens=True,
    remove_unused_columns=True,
):
    use_formatting_func = formatting_func is not None and dataset_text_field is None
    # self._dataset_sanity_checked = False

    # Inspired from: https://huggingface.co/learn/nlp-course/chapter7/6?fw=pt
    def tokenize(element):
        outputs = tokenizer(
            element[dataset_text_field] if not use_formatting_func else formatting_func(element),
            add_special_tokens=add_special_tokens,
            truncation=True,
            padding=False,
            max_length=max_seq_length,
            return_overflowing_tokens=False,
            return_length=False,
        )

        # if use_formatting_func and not self._dataset_sanity_checked:
        #     if not isinstance(formatting_func(element), list):
        #         raise ValueError(
        #             "The `formatting_func` should return a list of processed strings since it can lead to silent bugs."
        #         )
        #     else:
        #         self._dataset_sanity_checked = True

        return {"input_ids": outputs["input_ids"], "attention_mask": outputs["attention_mask"]}

    signature_columns = ["input_ids", "labels", "attention_mask"]

    extra_colmuns = list(set(dataset.column_names) - set(signature_columns))

    # if not remove_unused_columns and len(extra_colmuns) > 0:
    #     warnings.warn(
    #         "You passed `remove_unused_columns=False` on a non-packed dataset. This might create some issues with the default collator and yield to errors. If you want to "
    #         f"inspect dataset other columns (in this case {extra_colmuns}), you can subclass `DataCollatorForLanguageModeling` in case you used the default collator and create your own data collator in order to inspect the unused dataset columns."
    #     )

    tokenized_dataset = dataset.map(
        tokenize,
        batched=True,
        remove_columns=dataset.column_names if remove_unused_columns else None,
        num_proc=1,
        batch_size=32,
    )

    return tokenized_dataset

In [6]:
template = BASE_MISTRAL_TEMPLATE
sft_peft.templater = TemplateLLMQuerySender(None, template, start_seq='[', end_seq=']')
sft_peft.input_column = "basic_input"
sft_peft.target_column = "target_raw"

In [7]:
tokenized_dataset = _prepare_non_packed_dataloader(tokenizer, datasets['test'], None, None, format_prompt)

In [20]:
tokenized_dataset[]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 513
})

In [ ]:
collator(tokenized_dataset, return_tensors="np")

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from sft_peft import main, parse_args
import argparse


In [ ]:
args = parse_args([
    "--model", "mistralai/Mistral-7B-Instruct-v0.2",
    "--train-data", "../outputs/dataset_pipeline/fq17_3/fq17_3-split_train.pkl",
    "--target-column", "target_raw",
    "--input-column", "basic_input",
    "--valid-data", "../outputs/dataset_pipeline/fq17_3/fq17_3-split_valid.pkl",
    "--start-tag", "[query]",
    "--end-tag", "[/query]",
    "--rvalue", "32",
    "--lora-dropout", "0",
    "--batch-size", "1",
    "--gradient-accumulation", str(4),
    "--packing", "0",
    "--neft-tune-alpha", "0",
    "--epochs", "1",
    "--output", ".",
    "--save-name", "test_debug",
    "--run-name", f"test_debug",
    "--random-seed", "42"
])

In [ ]:
main(args)

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from transformers import AutoTokenizer


In [2]:
def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))

In [2]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.unk_token

In [3]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [12]:
print_tokens_with_ids("[/INST]")
print_tokens_with_ids("[/INST] [query]")
print_tokens_with_ids("[/INST] [query]")

[('▁[', 733), ('/', 28748), ('INST', 16289), (']', 28793)]
[('▁[', 733), ('/', 28748), ('INST', 16289), (']', 28793), ('▁[', 733), ('query', 3385), (']', 28793)]
[('▁[', 733), ('/', 28748), ('INST', 16289), (']', 28793), ('▁[', 733), ('query', 3385), (']', 28793)]


In [9]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset
from transformers import AutoTokenizer
from prompts_template import ELABORATE_INSTRUCTION, BASE_MISTRAL_TEMPLATE
from libwikidatallm.TemplateLLMQuerySender import TemplateLLMQuerySender

In [10]:
dataset = load_dataset("../datasets/final_queries_v1.7.json")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

print(ELABORATE_INSTRUCTION)

templater = TemplateLLMQuerySender(llm=None, template_text=BASE_MISTRAL_TEMPLATE, start_seq="[", end_seq="]")

dataset.head()

Your assignment involves a two-step process:
First, you will receive a task described in a single sentence. This task outlines what information you need to find or what question you need to answer.
Then, you will create a SPARQL Query: Based on the task given, you will write a SPARQL query. SPARQL is a specialized query language used to retrieve and manipulate data stored in Resource Description Framework (RDF) format. Your goal is to craft a query that, when executed, fetches the data or answers required by the initial instruction.
Make sure your SPARQL query is correctly formulated so that, upon execution, it produces the desired result matching the task's requirements.
Answer this following instruction:


,query,description,context,prompt,num_tokens,start_with_SELECT
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,<s>[INST] <<SYS>>This is a conversation betwee...,267,True
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,<s>[INST] <<SYS>>This is a conversation betwee...,296,True
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,<s>[INST] <<SYS>>This is a conversation betwee...,197,True
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,<s>[INST] <<SYS>>This is a conversation betwee...,203,True
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,<s>[INST] <<SYS>>This is a conversation betwee...,186,True


In [20]:
longest_query = dataset['query'].loc[dataset['query'].map(len).argmax()]

In [23]:
def prepare_prompts(query, templater:TemplateLLMQuerySender, template:str, start_tag:str = '`sparql\n', end_tag:str = "`"):
    start_tag = '`sparql\n'
    end_tag = "`"

    x = templater.apply_template({
        "system_prompt": template,
        "prompt": "\""+ ("abigword " * 20) +"\""
    })
    x += f"{start_tag}{query}{end_tag}"
    return x

longest_prompt = prepare_prompts(longest_query, templater=templater, template=ELABORATE_INSTRUCTION, start_tag="`sparql\n", end_tag="`")
len(tokenizer(longest_prompt)["input_ids"])

2739

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

import json
import random
from prompts_template import ELABORATE_INSTRUCTION

In [2]:
folder_path = Path("../tests/scriptstests/data/compute_metrics_real_data_from_training_1")
epochs_data = [json.loads((folder_path / f"Mistral-7B-Instruct-v0.2_rv32-ld0-bs1-ga4-gc1-p0-nta0-e3-ctx3072-q4bit-template-template-stsparql_compute_metrics_{i}.json").read_text()) for i in range(3)]

len(epochs_data)

3

In [3]:
columns = epochs_data[0].keys()
print(columns)

dict_keys(['preds', 'labels', 'decoded_labels', 'decoded_preds'])


In [4]:
def print_label_answer_and_preds(row_number:int = 0):
    print(f"LABEL n°{row_number}","=" * 98)
    print(epochs_data[0]["decoded_labels"][row_number])
    print("EPOCH 0","=" * 100)
    print(epochs_data[0]["decoded_preds"][row_number])
    print("EPOCH 1","-" * 100)
    print(epochs_data[1]["decoded_preds"][row_number])
    print("EPOCH 2","-" * 100)
    print(epochs_data[2]["decoded_preds"][row_number])

In [5]:
print(ELABORATE_INSTRUCTION)

Your assignment involves a two-step process:
First, you will receive a task described in a single sentence. This task outlines what information you need to find or what question you need to answer.
Then, you will create a SPARQL Query: Based on the task given, you will write a SPARQL query. SPARQL is a specialized query language used to retrieve and manipulate data stored in Resource Description Framework (RDF) format. Your goal is to craft a query that, when executed, fetches the data or answers required by the initial instruction.
Make sure your SPARQL query is correctly formulated so that, upon execution, it produces the desired result matching the task's requirements.
Answer this following instruction:


In [6]:
n_rows = len(epochs_data[0]["decoded_preds"])

print_label_answer_and_preds(random.randint(0, n_rows))

LABEL n°110 ==================================================================================================
`sparql
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>

SELECT (SUBSTR(STR(?item),32) AS ?wdq) (SUBSTR(STR(?prop),32) AS ?wdp) ?object ?url WHERE {
?item wdt:[property:collection] wd:[entity:Pitt Rivers Museum].
?item wdt:[property:described at URL] ?url FILTER CONTAINS(STR(?url), "ox.ac.uk").
?item ?p ?statement.
?prop wikibase:claim ?p; wikibase:statementProperty ?ps FILTER (?prop != wd:[property:described at URL]) FILTER (?prop != wd:[property:full work available at URL])
FILTER (?prop != wd:[property:image]) FILTER (?prop != wd:[property:inception]).
?statement ?ps ?object .
FILTER NOT EXISTS {?statement prov:wasDerivedFrom ?derivedFrom}
} ORDER BY ?item`
EPOCH 0 ============================================================================

In [19]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, load_peft_weights
from prompts_template import ELABORATE_INSTRUCTION, BASE_MISTRAL_TEMPLATE
import torch
import os

In [3]:
model_path = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_path = "../outputs/batch_run/learning_experiment_long_epochs_4/models/M-7B-I-v0.2_oawtor-rv256-rm1-ld0-bs2-ga8-gc1-p0-nta0-e10-ctx3072-qno-template-template-stsparql_adapters"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device,
    torch_dtype=dtype,
)

try:
    model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)
except RuntimeError as e: # RuntimeError => Might be padding token is left in the embedding when saved
    d_adpt = load_peft_weights(adapter_path)
    len_d_adpt = len(d_adpt.get('base_model.model.lm_head.weight', 0))
    if len_d_adpt != 0 and len_d_adpt != len(tokenizer):
        model.resize_token_embeddings(len(tokenizer) + 1)
        model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)
    else:
        raise e
except Exception as e:
    try:
        model = PeftModel.from_pretrained(model, os.path.abspath(adapter_path), is_trainable=False)
    except RuntimeError as e: # RuntimeError => Might be padding token is left in the embedding when saved
        d_adpt = load_peft_weights(adapter_path)
        len_d_adpt = len(d_adpt.get('base_model.model.lm_head.weight', 0))
        if len_d_adpt != 0 and len_d_adpt != len(tokenizer):
            model.resize_token_embeddings(len(tokenizer) + 1)
            model = PeftModel.from_pretrained(model, os.path.abspath(adapter_path), is_trainable=False)
        else:
            raise e
    except Exception as e:
        raise e

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
d_adpt = load_peft_weights(adapter_path)
len(d_adpt.get('base_model.model.lm_head.weight', 0))

32001

In [25]:
list(d_adpt.keys())

['base_model.model.lm_head.weight',
 'base_model.model.model.embed_tokens.weight',
 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight',
 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight',
 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight',
 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight',
 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.weight',
 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.weight',
 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.weight',
 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.weight',
 'base_model.model.model.layers.10.self_attn.q_proj.lora_A.weight',
 'base_model.model.model.layers.10.self_attn.q_proj.lora_B.weight',
 'base_model.model.model.layers.10.self_attn.v_proj.lora_A.weight',
 'base_model.model.model.layers.10.self_attn.v_proj.lora_B.weight',
 'base_model.model.model.layers.11.self_attn.q_proj.lora_A.weight',
 'base_model.model.model.layers.11.self_a

In [14]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32000, 4096)

In [8]:
modified_tokenizer = AutoTokenizer.from_pretrained(model_path)
modified_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [12]:
print(f"{modified_tokenizer.pad_token=}")
print(f"{modified_tokenizer.pad_token_id=}")

modified_tokenizer.pad_token='[PAD]'
modified_tokenizer.pad_token_id=32000


In [17]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset
from transformers import AutoTokenizer
from generate_finetune_dataset import keep_working_queries
import random

In [2]:
path = Path("../outputs/dataset_pipeline/fq17_3/fq17_3-generated_prompt-executed.parquet.gzip")
if not path.exists():
    raise FileExistsError(f"No file was found at this path: {path}")

dataset = load_dataset(path)
dataset.head()

,query,description,context,prompt,num_tokens,start_with_SELECT,query_templated,basic_prompt,basic_num_tokens,basic_result,...,basic_is_skipped,basic_is_prompt_too_long,templated_prompt,templated_num_tokens,templated_result,templated_full_answer,templated_is_skipped,templated_is_prompt_too_long,execution,executed_query
index,,,,,,,,,,,,,,,,,,,,,
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,<s>[INST] <<SYS>>This is a conversation betwee...,267,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"<s>[INST] QUERY=""SELECT ?property ?propertyTyp...",226,"[""Write a SparQL query to retrieve Wikidata pr...",...,False,False,"<s>[INST] QUERY=""PREFIX bd: <http://www.bigdat...",289,"[""Write a SparQL query to retrieve all Wikidat...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,"[{'property': {'type': 'uri', 'value': 'http:/...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,<s>[INST] <<SYS>>This is a conversation betwee...,296,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"<s>[INST] QUERY=""SELECT ?id ?idLabel ?idDescri...",259,"[""Write a SparQL query to retrieve the ID, lab...",...,False,False,"<s>[INST] QUERY=""PREFIX bd: <http://www.bigdat...",322,"[""Write a SparQL query to retrieve the ID, lab...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,"[{'id': {'type': 'uri', 'value': 'http://www.w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,<s>[INST] <<SYS>>This is a conversation betwee...,197,True,PREFIX wd: <http://www.wikidata.org/entity/>\n...,"<s>[INST] QUERY=""SELECT (COUNT(?article) AS ?c...",159,"[""Find the total number of scientific articles...",...,False,False,"<s>[INST] QUERY=""PREFIX wd: <http://www.wikida...",201,"[""Write a SparQL query to retrieve the count o...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,timeout,PREFIX wd: <http://www.wikidata.org/entity/>\n...
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,<s>[INST] <<SYS>>This is a conversation betwee...,203,True,PREFIX wd: <http://www.wikidata.org/entity/>\n...,"<s>[INST] QUERY=""SELECT (COUNT(DISTINCT ?artic...",165,"[""Find the number of distinct fictional charac...",...,False,False,"<s>[INST] QUERY=""PREFIX wd: <http://www.wikida...",210,"[""Write a query to find the total number of di...","{'content': ' 1. ""Write a query to find the to...",False,False,[{'count': {'datatype': 'http://www.w3.org/200...,PREFIX wd: <http://www.wikidata.org/entity/>\n...
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,<s>[INST] <<SYS>>This is a conversation betwee...,186,True,PREFIX wdt: <http://www.wikidata.org/prop/dire...,"<s>[INST] QUERY=""SELECT (COUNT(?item) AS ?coun...",148,"[""How many items in Wikidata have recorded coo...",...,False,False,"<s>[INST] QUERY=""PREFIX wdt: <http://www.wikid...",172,"[""Write a SparQL query to retrieve the number ...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,[{'count': {'datatype': 'http://www.w3.org/200...,PREFIX wdt: <http://www.wikidata.org/prop/dire...


In [3]:
dataset.keys()

Index(['query', 'description', 'context', 'prompt', 'num_tokens',
       'start_with_SELECT', 'query_templated', 'basic_prompt',
       'basic_num_tokens', 'basic_result', 'basic_full_answer',
       'basic_is_skipped', 'basic_is_prompt_too_long', 'templated_prompt',
       'templated_num_tokens', 'templated_result', 'templated_full_answer',
       'templated_is_skipped', 'templated_is_prompt_too_long', 'execution',
       'executed_query'],
      dtype='object')

In [4]:
dropped_dataset = keep_working_queries(dataset)

columns_to_take = [
    "templated_result",
    "query_templated",
]

dataset_for_tokenization = dropped_dataset[columns_to_take]
dataset_for_tokenization.head()

,templated_result,query_templated
index,,
0,"[""Write a SparQL query to retrieve all Wikidat...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1,"[""Write a SparQL query to retrieve the ID, lab...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
3,"[""Write a query to find the total number of di...",PREFIX wd: <http://www.wikidata.org/entity/>\n...
4,"[""Write a SparQL query to retrieve the number ...",PREFIX wdt: <http://www.wikidata.org/prop/dire...
5,"[""Write a SparQL query to retrieve the number ...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...


In [5]:
all_concat_dataset = dataset_for_tokenization.apply(lambda x: x['templated_result'][0].strip() + x["query_templated"].strip(), axis=1).sum()
print(f"{len(all_concat_dataset)=}")

len(all_concat_dataset)=2294959


In [6]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
tokens = tokenizer(all_concat_dataset)
type(tokens)

transformers.tokenization_utils_base.BatchEncoding

In [8]:
print(f"{len(tokens.tokens())=}")

len(tokens.tokens())=824341


In [27]:
entry_iloc_id = random.randint(0, len(dropped_dataset) - 1)
entry = dropped_dataset.iloc[entry_iloc_id]
print(f"Entry n°{entry_iloc_id}")
print("----- Prompts From Original -----")
print("\n".join(entry["basic_result"]))
print("----- Original Query -----")
print(entry["query"], "\n")
print("----- Prompts From Templated -----")
print("\n".join(entry["templated_result"]))
print("----- Templated Query -----")
print(entry["query_templated"])

Entry n°353
----- Prompts From Original -----
"Write a SPARQL query to retrieve all books with a title, illustrator, publisher, and publication date, but allow for the possibility that some books may not have an illustrator or publisher listed."
"Create a SPARQL query to find all books with a title, optional illustrator and publisher information, and a publication date."
"Generate a SPARQL query to retrieve books with required title, publication date, and optional illustrator and publisher information."
----- Original Query -----
SELECT ?book ?title ?illustratorLabel ?publisherLabel ?published
WHERE
{
?book wdt:P50 wd:Q35610. # required wdt:P50
OPTIONAL { # match all or none from group:
?book wdt:P1476 ?title; # required wdt:P1476
wdt:P110 ?illustrator; # required wdt:P110
wdt:P123 ?publisher; # required wdt:P123
wdt:P577 ?published. # required wdt:P577
}
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
} 

----- Prompts From Templated -----
"Write a S